
# Prediction of unsteady transonic buffet flow field

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Introduction

At transonic flow conditions, the self-sustained large-scale oscillation of shock wave on airfoils is called transonic buffet. The reason is related to the flow separation and interaction between shock wave and boundary layer. After entering the buffet boundary, the change of separation zone induces the flow instability and affects the position of shock wave, which makes the shock wave move forward and backward and comes with complex unsteady and nonlinear characteristics. Learning the unsteady shock buffet flow directly from the flow field (spatial-temporal flow characteristics) is a valuable and challenging method for buffet research. In order to find an efficient DL modeling method for the complex unsteady transonic buffet flow, an enhanced hybrid deep neural network (eHDNN) is designed to predict the unsteady flow field based on flow field reconstruction.

![p1.png](./images/p1.png)

## Framework of eHDNN

The basic framework of the eHDNN is mainly based on the hybrid deep neural network framework, which is constituted by CNN, ConvLSTM and DeCNN.CNN reduces the dimensionality of time series flow fields and realizes the characteristic extraction; ConvLSTM learns the evolution of low-dimensional spatial-temporal characteristics and make prediction; finally, DeCNN achieves the reconstruction of predicted flow field characteristics

+ Input layer: inputting the historical flow fields
+ Convolutional layer: reducing the dimensionality of flow fields and extract the high-dimensional spatial-temporal flow characteristics by CNN
+ Memory layer: learning the evolution of the spatial-temporal characteristics of flow fields in the low-dimensional space and predicting the next moment by ConvLSTM
+ Deconvolutional output layer: restoring the predicted low-dimensional characteristics of flow fields to high-dimensional space to achieve the reconstruction of the prediction of transient flow field at the next moment by DeCNN, then outputting the prediction

![p2.jpg](./images/p2.jpg)

## Training samples

Training samples are constructed from multi-dimensional matrix flow field snapshot matrixs obtained from numerical simulation flow field data of unsteady buffet of OAT15A supercritical airfoil. It is divided into single-state data set and multi-state data set

+ The multi-state data set is the sequence data of unsteady buffet flow fields under the condition of multiple angles of attack. The angles of attack include 3.3°, 3.4°, 3.5°, 3.6°, 3.7° and 3.8°, all of which are located within the buffet boundary
+ The single-state data set is the sequence data of unsteady buffet flow fields under the condition of single angle of attack, which is any of the above angles (3.5° by default). Each flow field snapshot contains 3 channels, representing pressure distribution information, string velocity information and normal velocity information of flow field. The size of multidimensional matrix flow field snapshot matrix is T×C×H×W(C=3,H=200,W=200,C is the number of channels,H and W are respectively the height and width of the snapshot)

In [1]:
import os
import time
import argparse
import urllib
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, jit, data_sink, set_seed
from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
from mindflow.utils import load_yaml_config

In [2]:
from src import create_dataset, ForwardNet, HybridLoss

## Training environment

+ The static GRAPH of Mindspore framework is adopted for training
+ Training can be done on GPU (default) or Ascend (single card)
+ There are two differences between single-state and multi-state training conditions: 1) different training data sets; 2) Different depth of memory layer: The memory layer in single-state is 2 layers (num_memory_layers=2); the memory layer in multi-state is 4 layers (num_memory_layers=4)

In [3]:
set_seed(0)
np.random.seed(0)

In [4]:
parser = argparse.ArgumentParser(description='eHDNN for Transonic buffet')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--train_aoa_list", type=list, default=[35],
                    help="The type for training, [33 ,34 , 35 , 36 , 37 , 38] for multi_state training /n"
                         "[33],....,[38] for single_state training")
parser.add_argument("--num_memory_layers", type=int, default=2, choices=[2, 4],
                    help="The number of layers of the whole Memory layer， 2 in single_state and 4 in multi state")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args(args=[])
context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## Training hyperparameter

In [5]:
config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## Path for saving training process files

In [6]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Construct neural network

In [7]:
model = ForwardNet(model_params["in_channels"],
                   model_params["out_channels"],
                   model_params["num_layers"],
                   args.num_memory_layers,
                   model_params["kernel_size_conv"],
                   model_params["kernel_size_lstm"])

## Optimizer

In [8]:
loss_func = HybridLoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## Training framework

In [9]:
def forward_fn(x, y):
    pred = model(x)
    loss = loss_func(pred, y)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)
@jit
def train_step(x, y):
    loss, grads = grad_fn(x, y)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss
@jit
def eval_step(x, y):
    loss = forward_fn(x, y)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
    return loss

## Dataset loading

In [10]:
print(f"======Load data sample ========")
data_dir = data_params["data_dir"]
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
url_1 = "https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/airfoil/2D_unsteady/"
for aoa in args.train_aoa_list:
    url_2 = f"total{aoa}_puv.mat"
    url_aoa = urllib.parse.urljoin(url_1, url_2)
    urllib.request.urlretrieve(url_aoa, data_dir + '/' + url_2)
dataset_train, dataset_eval = create_dataset(data_dir,
                                             data_params["data_length"],
                                             data_params["train_ratio"],
                                             args.train_aoa_list)
print(f"==========End Load=============")

======Load data sample ========
input shape (2183, 16, 3, 200, 200)
label shape (2183, 1, 3, 200, 200)
==========End Load=============


## Data sink operation

In [11]:
train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

## Training

In [12]:
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    model.set_train(True)
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(train_sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")
    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")
        model.set_train(False)
        eval_loss = []
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            eval_loss.append(step_eval_loss)
        epoch_eval_loss = sum(eval_loss) / len(eval_loss)
        print(f"epoch: {epoch} eval loss: {epoch_eval_loss}")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")
    print(f"=====================End train========================")

pid:22673
========Start train============
epoch: 1 train loss: 1.3850026 epoch time: 317.26s
epoch: 2 train loss: 0.70951277 epoch time: 300.97s
epoch: 3 train loss: 0.6354316 epoch time: 314.41s
epoch: 4 train loss: 0.599306 epoch time: 309.21s
epoch: 5 train loss: 0.5684712 epoch time: 311.57s
epoch: 6 train loss: 0.54864025 epoch time: 306.30s
epoch: 7 train loss: 0.5113489 epoch time: 304.56s
epoch: 8 train loss: 0.34839326 epoch time: 291.30s
epoch: 9 train loss: 0.2967865 epoch time: 297.61s
epoch: 10 train loss: 0.28633794 epoch time: 298.61s
======Start Evaluation=========
epoch: 10 eval loss: 0.33136454
=======End Evaluation==========
=========End train=============


## Visualization of predicted flow field results

+ run prediction.py
+ The following figures show the prediction results of unsteady buffet flow fields in a single period under the angle of attack of 3.75° (generalized state) based on the well-trained eHDNN model (pressure field)

<figure class="harf">
    <img src="./images/375_pressure_cfd.gif" title="cfd" width="200"/>
    <img src="./images/375_pressure_prediction.gif" title="prediction" width="200"/>
    <img src="./images/375_pressure_abserror.gif" title="abs error" width="200"/>
</center>